In [114]:
# Import the necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os


In [115]:
# first set the working directory. This code will be changed based on the relative location of the data files 
# on the local drive of the computer executing the command. 
os.chdir('C:\\Users\\belincoln\\Documents\\! CBP\\!User Fees\\!! Goal 1 Dashboards')

# Remember to set the working directory to whatever folder you have saved the zip file to. 
# Works well for Jupyter Notebooks, can be configured in IDE using file explorer. 
collections = pd.read_excel(os.path.join('Source Emails & Source Files','Files','Collections','COBRA_Air','Collections cc495 - FY13 - FY18.xls'))

In [116]:
# Begin Data Cleaning
# delete columns and rows that contain only na
collections = collections.dropna(axis=0, how = 'all')
collections = collections.dropna(axis=1, how = 'all')

In [118]:
# Delete the first two rows and only keep the Period Column and Collection Columns
collections = collections.iloc[2:,[1,-1,-3,-4]]

In [119]:
collections

,Unnamed: 1,Unnamed: 17,Unnamed: 15,Unnamed: 14
7,NaN,Applied Penalty Amount,Applied Interest Amount,Applied Principal Amount
8,Period,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN
12,Qtr 02 (Apr-Jun) 2013,0,0,5.5
14,Qtr 03 (Jul-Sept) 2012,0,0,5.5
...,...,...,...,...
25318,Qtr 04 (Oct-Dec) 2012,0,0,626
25320,Qtr 04 (Oct-Dec) 2013,0,0,4917
25322,NaN,0,0,34902.5
25324,NaN,0,774970,3.57479e+09


In [120]:
# Make first row column headers
collections.columns = collections.iloc[0,:]

In [121]:
# Make sure all columns have a name! Rename first column 'Period'
# Delete first two rows
collections.columns.values[0] = 'Period'
#collections.columns.values[-1] = 'Class Code'
collections = collections.iloc[2:,:]

In [122]:
# Delete rows that contain sums for each company (don't want to double count collections)
# Also delete additional NaNs
collections = collections.dropna(axis = 0, subset = ['Period'])

In [123]:
# Sum Interest, Penalty, and Principal Collection Amounts
collections['Collections'] = collections[['Applied Penalty Amount', 'Applied Interest Amount',
       'Applied Principal Amount']].sum(axis = 1)

In [124]:
# Groups Collections based on Collection Period, across all companies
collections = collections.groupby(collections['Period']).sum()

In [125]:
# Remove audit payments
collections= collections[~collections.index.str.contains("\*")]


# Add an additional column that shows remittance period (independent of year)
collections['Remittance Period'] = collections.index.str.split('20').str[0]


# Create Calendar Year Column
#collections['Calendar Year'] = collections.index.str.split(')').str[1]
# Turn Years into integers
#collections['Calendar Year'] = collections['Calendar Year'].astype(int)

# Filter on years not a part of analysis
#years = [2012,2013,2014,2015,2016,2017,2018]
#collections = collections[collections['Calendar Year'].isin(years)]

In [126]:
collections.index.unique()

Index(['Qtr 01 (Jan-Mar) 2006 ', 'Qtr 01 (Jan-Mar) 2007 ',
       'Qtr 01 (Jan-Mar) 2008 ', 'Qtr 01 (Jan-Mar) 2009 ',
       'Qtr 01 (Jan-Mar) 2010 ', 'Qtr 01 (Jan-Mar) 2011 ',
       'Qtr 01 (Jan-Mar) 2012 ', 'Qtr 01 (Jan-Mar) 2013 ',
       'Qtr 01 (Jan-Mar) 2014 ', 'Qtr 01 (Jan-Mar) 2015 ',
       'Qtr 01 (Jan-Mar) 2016 ', 'Qtr 01 (Jan-Mar) 2017 ',
       'Qtr 01 (Jan-Mar) 2018 ', 'Qtr 01 (Jan-Mar) 2019 ',
       'Qtr 02 (Apr-Jun) 2004 ', 'Qtr 02 (Apr-Jun) 2005 ',
       'Qtr 02 (Apr-Jun) 2007 ', 'Qtr 02 (Apr-Jun) 2008 ',
       'Qtr 02 (Apr-Jun) 2009 ', 'Qtr 02 (Apr-Jun) 2010 ',
       'Qtr 02 (Apr-Jun) 2011 ', 'Qtr 02 (Apr-Jun) 2012 ',
       'Qtr 02 (Apr-Jun) 2013 ', 'Qtr 02 (Apr-Jun) 2014 ',
       'Qtr 02 (Apr-Jun) 2015 ', 'Qtr 02 (Apr-Jun) 2016 ',
       'Qtr 02 (Apr-Jun) 2017 ', 'Qtr 02 (Apr-Jun) 2018 ',
       'Qtr 02 (Apr-Jun) 2019 ', 'Qtr 03 (Jul-Sept) 2007 ',
       'Qtr 03 (Jul-Sept) 2008 ', 'Qtr 03 (Jul-Sept) 2009 ',
       'Qtr 03 (Jul-Sept) 2010 ', 'Qtr 03 (Jul-Sept) 

In [127]:
collections.loc['Qtr 03 (Jul-Sept) 2016 ~']

7
Collections                          66
Remittance Period    Qtr 03 (Jul-Sept) 
Name: Qtr 03 (Jul-Sept) 2016 ~, dtype: object

In [128]:
collections

7,Collections,Remittance Period
Period,,
Qtr 01 (Jan-Mar) 2006,1.749100e+02,Qtr 01 (Jan-Mar)
Qtr 01 (Jan-Mar) 2007,2.362590e+03,Qtr 01 (Jan-Mar)
Qtr 01 (Jan-Mar) 2008,1.506374e+04,Qtr 01 (Jan-Mar)
Qtr 01 (Jan-Mar) 2009,2.619046e+04,Qtr 01 (Jan-Mar)
Qtr 01 (Jan-Mar) 2010,1.169274e+05,Qtr 01 (Jan-Mar)
Qtr 01 (Jan-Mar) 2011,9.325381e+04,Qtr 01 (Jan-Mar)
Qtr 01 (Jan-Mar) 2012,2.878609e+05,Qtr 01 (Jan-Mar)
Qtr 01 (Jan-Mar) 2013,1.446142e+08,Qtr 01 (Jan-Mar)
Qtr 01 (Jan-Mar) 2014,1.519669e+08,Qtr 01 (Jan-Mar)


In [129]:
collections['Collections'] = collections[['Applied Penalty Amount', 'Applied Interest Amount',
       'Applied Principal Amount']].sum(axis = 1)

KeyError: "None of [Index(['Applied Penalty Amount', 'Applied Interest Amount',\n       'Applied Principal Amount'],\n      dtype='object', name=7)] are in the [columns]"

In [ ]:
collections.index.name
#collections.reset_index().replace({'index': {'Uncoded & errors':'Other / unknown'}}.groupby('index', sort=False).sum()